In [1]:
import argparse
import numpy as np
import pandas as pd
from multiprocessing import cpu_count
from sklearn.model_selection import train_test_split
import re
import os
import datetime

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [3]:
import matplotlib.pyplot as plt

In [4]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [5]:
%load_ext tensorboard

In [6]:
window = 5 # depends on time window
epochs = 100
batch_size = 16
pose_vec_dim = 36 # depends on pose estimation model used

In [7]:
class_names = [0, 1, 2]
num_class = len(class_names)
lbl_dict = {class_name:idx for idx, class_name in enumerate(class_names)}

In [51]:
csv_path = './data/csv/result.csv'
validate_csv_path = './data/csv/validate.csv'

In [52]:
dataset = pd.read_csv(csv_path,  index_col=None)

In [53]:
def convert_vec(x):
    return np.array(re.split(',*',re.sub('^\D\D*|\D*\D$', '',re.sub('\D', ',', str(x)))), dtype=float)

In [54]:
def shape_data(X, Y, window):
    new_x = []
    new_y = []
    max_row = X.shape[0] - (X.shape[0] % window + 1)
    i = window
    while i < max_row:
        new_x.append(X[i-window+1:i+1]-X[i-window:i])
        new_y.append(Y[i])
        i+=1
    return np.array(new_x), np.array(new_y)

In [56]:
dataset = pd.read_csv(csv_path,  index_col=None)

y = dataset.label.values
X = np.stack(dataset.vec.apply(convert_vec).values)

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [57]:
min_max_scaler = MinMaxScaler().fit(X)

X = min_max_scaler.transform(X)

X,y = shape_data(X,y,window)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

y_train = tf.keras.utils.to_categorical(list(map(lbl_dict.get, y_train)), num_class)
y_test = tf.keras.utils.to_categorical(list(map(lbl_dict.get, y_test)), num_class)

#y_valid = y_test[:y_test.shape[0] // 3]
#X_valid = X_test[:X_test.shape[0] // 3]

#y_test = y_test[y_test.shape[0] // 3:]
#X_test = X_test[X_test.shape[0] // 3:]

#X_test = X_test.reshape(X_test.shape[0], pose_vec_dim, window)
#X_train = X_train.reshape(X_train.shape[0], pose_vec_dim, window)

In [83]:
X_train.shape

(5931, 5, 36)

In [58]:
import pickle
with open('min_max_scaler.pickle', 'wb') as f:
    pickle.dump(min_max_scaler, f)

In [84]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
checkpointer = ModelCheckpoint(filepath="../app/models/lstm_model.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)

In [89]:
model = Sequential()
model.add(LSTM(32, recurrent_dropout=0.5, input_shape=(window, pose_vec_dim)))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(class_names), activation='softmax'))
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 32)                8832      
_________________________________________________________________
dense_22 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_14 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 3)                 51        
Total params: 9,411
Trainable params: 9,411
Non-trainable params: 0
_________________________________________________________________
None


In [90]:
model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

In [91]:
%tensorboard --logdir logs --port 6006 --bind_all

In [ ]:
history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(X_test, y_test),
                        callbacks=[checkpointer, tensorboard_callback])

Epoch 1/100
371/371 [==============================] - 12s 28ms/step - loss: 1.0267 - accuracy: 0.4709 - val_loss: 0.9487 - val_accuracy: 0.5576

Epoch 00001: val_loss did not improve from 0.47671
Epoch 2/100
371/371 [==============================] - 9s 25ms/step - loss: 0.9377 - accuracy: 0.5827 - val_loss: 0.8770 - val_accuracy: 0.6138

Epoch 00002: val_loss did not improve from 0.47671
Epoch 3/100
371/371 [==============================] - 9s 25ms/step - loss: 0.8698 - accuracy: 0.6220 - val_loss: 0.8219 - val_accuracy: 0.6563

Epoch 00003: val_loss did not improve from 0.47671
Epoch 4/100
371/371 [==============================] - 9s 25ms/step - loss: 0.8263 - accuracy: 0.6522 - val_loss: 0.7737 - val_accuracy: 0.6736

Epoch 00004: val_loss did not improve from 0.47671
Epoch 5/100
371/371 [==============================] - 9s 25ms/step - loss: 0.7920 - accuracy: 0.6684 - val_loss: 0.7523 - val_accuracy: 0.6783

Epoch 00005: val_loss did not improve from 0.47671
Epoch 6/100
371/371

371/371 [==============================] - 9s 24ms/step - loss: 0.4011 - accuracy: 0.8449 - val_loss: 0.4751 - val_accuracy: 0.8313

Epoch 00042: val_loss did not improve from 0.47384
Epoch 43/100
371/371 [==============================] - 9s 25ms/step - loss: 0.3887 - accuracy: 0.8535 - val_loss: 0.4697 - val_accuracy: 0.8250

Epoch 00043: val_loss improved from 0.47384 to 0.46969, saving model to ../app/models/lstm_model.h5
Epoch 44/100
371/371 [==============================] - 9s 25ms/step - loss: 0.3884 - accuracy: 0.8537 - val_loss: 0.4654 - val_accuracy: 0.8364

Epoch 00044: val_loss improved from 0.46969 to 0.46542, saving model to ../app/models/lstm_model.h5
Epoch 45/100
371/371 [==============================] - 9s 25ms/step - loss: 0.3785 - accuracy: 0.8550 - val_loss: 0.4798 - val_accuracy: 0.8282

Epoch 00045: val_loss did not improve from 0.46542
Epoch 46/100
371/371 [==============================] - 9s 25ms/step - loss: 0.3704 - accuracy: 0.8624 - val_loss: 0.4660 - val

371/371 [==============================] - 9s 25ms/step - loss: 0.2584 - accuracy: 0.9064 - val_loss: 0.4630 - val_accuracy: 0.8608

Epoch 00081: val_loss did not improve from 0.44750
Epoch 82/100
371/371 [==============================] - 9s 25ms/step - loss: 0.2669 - accuracy: 0.9004 - val_loss: 0.4621 - val_accuracy: 0.8557

Epoch 00082: val_loss did not improve from 0.44750
Epoch 83/100
371/371 [==============================] - 9s 25ms/step - loss: 0.2691 - accuracy: 0.9024 - val_loss: 0.4721 - val_accuracy: 0.8580

Epoch 00083: val_loss did not improve from 0.44750
Epoch 84/100
371/371 [==============================] - 9s 25ms/step - loss: 0.2686 - accuracy: 0.9025 - val_loss: 0.4667 - val_accuracy: 0.8584

Epoch 00084: val_loss did not improve from 0.44750
Epoch 85/100
371/371 [==============================] - 9s 25ms/step - loss: 0.2621 - accuracy: 0.9007 - val_loss: 0.4676 - val_accuracy: 0.8514

Epoch 00085: val_loss did not improve from 0.44750
Epoch 86/100
371/371 [======

In [41]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Ошибка')
plt.ylabel('Ошибка')
plt.xlabel('Эпоха')
plt.legend(['Обучающая выборка', 'Тестовая выборка'], loc='best')
plt.show()

plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Точность')
plt.ylabel('Точность')
plt.xlabel('Эпоха')
plt.legend(['Обучающая выборка', 'Тестовая выборка'], loc='best')
plt.show()

NameError: name 'history' is not defined

<Figure size 432x288 with 0 Axes>

In [42]:
dataset = pd.read_csv(validate_csv_path,  index_col=None)

y_valid = dataset.label.values
X_valid = np.stack(dataset.vec.apply(convert_vec).values)

X_valid = min_max_scaler.transform(X_valid)

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [43]:
X_valid,y_valid = shape_data(X_valid,y_valid,window)

y_valid = tf.keras.utils.to_categorical(list(map(lbl_dict.get, y_valid)), num_class)

In [49]:
model = tf.keras.models.load_model('../app/models/gru_model.h5')

In [78]:
%%time
model.evaluate(X_valid, y_valid)

88/88 [==============================] - 1s 6ms/step - loss: 23.7980 - accuracy: 0.3731
CPU times: user 697 ms, sys: 61.5 ms, total: 759 ms
Wall time: 598 ms


[23.79796028137207, 0.3731343150138855]

In [47]:
from sklearn.metrics import f1_score

In [79]:
model_lstm = tf.keras.models.load_model('../app/models/lstm_model.h5')
model_gru = tf.keras.models.load_model('../app/models/gru_model.h5')
lstm_predict = np.argmax(model_lstm.predict(X_valid), 1)
gru_predict = np.argmax(model_gru.predict(X_valid), 1)

In [80]:
print("Gru f1-score: {0}".format(f1_score(np.argmax(y_valid, 1), lstm_predict, average='weighted')))
print("LSTM f1-score: {0}".format(f1_score(np.argmax(y_valid, 1), gru_predict, average='weighted')))

Gru f1-score: 0.3520339713449763
LSTM f1-score: 0.35029539354403627


In [ ]:
y_valid.shape